In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns 
import sklearn.preprocessing as preprocessing 
import sklearn.preprocessing as OneHotEncoder
import sklearn.preprocessing as LabelEncoder
import sklearn.metrics as metrics 
import sklearn.calibration as calibration 
import sklearn.linear_model as linear_model 
import sklearn.svm as svm 
import sklearn.naive_bayes as naive_bayes
import sklearn.model_selection as model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

In [ ]:
#df_chunk = pd.read_csv("/kaggle/input/save-the-attack-contest/train_data.csv",)   
#df=df_chunk.get_chunk(100000)
df = pd.read_csv("/kaggle/input/save-the-attack-contest/train_data.csv")

In [ ]:
df_test = pd.read_csv("/kaggle/input/save-the-attack-contest/test_data.csv")

In [ ]:
df_test.shape

In [ ]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
#df = reduce_mem_usage(df)

In [ ]:
#pd.set_option('max_rows', 999)
#df.isna().sum()


In [ ]:
#columns_list = []
#columns_list=df.columns
#print(columns_list)

In [ ]:
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
categorical_cols = df.select_dtypes('object').columns.tolist()

In [ ]:
print(len(numeric_cols))
print(len(categorical_cols))
print(df.columns.shape)


In [ ]:
numeric_cols = df_test.select_dtypes(include=np.number).columns.tolist()
categorical_cols = df_test.select_dtypes('object').columns.tolist()

In [ ]:
df_together = pd.concat([df,df_test],axis = 0)

In [ ]:
df_together = reduce_mem_usage(df_together)

In [ ]:
df_together.columns

In [ ]:
#df[["Census_IsTouchEnabled", "HasDetections"]].groupby(["Census_IsTouchEnabled"], as_index=False).mean().sort_values(by="HasDetections", ascending=False)
#df[["Census_IsPenCapable", "HasDetections"]].groupby(["Census_IsPenCapable"], as_index=False).mean().sort_values(by="HasDetections", ascending=False)
#df[["Census_IsAlwaysOnAlwaysConnectedCapable", "HasDetections"]].groupby(["Census_IsAlwaysOnAlwaysConnectedCapable"], as_index=False).mean().sort_values(by="HasDetections", ascending=False)
#df[["Census_IsAlwaysOnAlwaysConnectedCapable", "HasDetections"]].groupby(["Census_IsAlwaysOnAlwaysConnectedCapable"], as_index=False).mean().sort_values(by="HasDetections", ascending=False)


In [ ]:
#(df[numeric_cols].isna().sum()/df.shape[0])*100
#len(numeric_cols)

In [ ]:
null_list=['DefaultBrowsersIdentifier','Census_IsWIMBootEnabled','Census_ThresholdOptIn','Census_IsFlightingInternal']
for i in null_list:
    numeric_cols.remove(i)

In [ ]:
(df_together[categorical_cols].isna().sum()/df_together.shape[0])*100

In [ ]:
null_list2 = ['PuaMode','Census_ProcessorClass','Census_InternalBatteryType','SmartScreen']
for i in null_list2:
    categorical_cols.remove(i)
print(len(categorical_cols))

In [ ]:
categorical_cols

In [ ]:
(df_together[numeric_cols].isna().sum()/df_together.shape[0])*100

In [ ]:
for i in numeric_cols:
    df_together[i].fillna(df[i].mean(),inplace =True)
    

In [ ]:
df_together.isna().sum()

In [ ]:
df_c = df_together[categorical_cols]

In [ ]:
df_c = df_c.drop(columns = 'MachineIdentifier')

In [ ]:
#df_c.nunique()

In [ ]:
#3for i in categorical_cols[1:]:
    #df_c[i].fillna(df_c[i].mode(),inplace =True)

In [ ]:
#df_c.isna().sum()

In [ ]:
#df_c['SmartScreen'].fillna(df_c['SmartScreen'].mode(),inplace = True) 

In [ ]:
df_c = df_c.fillna(df_c.mode().iloc[0])

In [ ]:

#df_c['SmartScreen'].isna().sum()

In [ ]:
df_c.isna().sum()

In [ ]:
df_c

In [ ]:
df_together[numeric_cols] = reduce_mem_usage(df_together[numeric_cols])

In [ ]:
df_c= reduce_mem_usage(df_c)

In [ ]:
df_c.nunique()

In [ ]:
#df_c["ProductName"].nunique(), len(df_c.columns)

In [ ]:
l = df_c.columns

In [ ]:
#unique_list=df_c.nunique()

In [ ]:
#def encoding(x,cat_list,unique_list):
    #label_list = []
    #one_hot_list = []
    #for i in range(len(cat_list)):
        #if unique_list[i] >= x:
            #label_list.append(cat_list[i])
        #else:
             #one_hot_list.append(cat_list[i])
                
            
    #return label_list, one_hot_list    
            
        #else:
        #    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
        #    X = onehotencoder.fit_transform(data.l[i].values.reshape(-1,1)).toarray()
        #    dfOneHot = pd.DataFrame(X) 
        #    df = pd.concat([data, dfOneHot], axis=1)
        #    #droping the country column 
        #    df= df.drop([l[i]], axis=1)
            

            

            

In [ ]:
#l_list,h_list = encoding(100,l,unique_list)

In [ ]:
l_list = l

In [ ]:
#encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
#encoder.fit(df_c[h_list])
#encoded_cols = list(encoder.get_feature_names(h_list))
#df_c[encoded_cols] = encoder.transform(df_c[h_list])

In [ ]:
lencoder = preprocessing.LabelEncoder()
for i in l_list:
    df_c[i] = lencoder.fit_transform(df_c[i])

In [ ]:
df_c.dtypes

In [ ]:
df_c[encoded_cols].shape

In [ ]:
df_c.astype('int64')

In [ ]:
df_c.dtypes

In [ ]:
df_c = reduce_mem_usage(df_c)

In [ ]:
#df_c[h_list].nunique().sum()

In [ ]:
df_c =df_c.drop(columns = h_list)

In [ ]:
df_c.shape

In [ ]:
df_c[l_list]

In [ ]:
df[numeric_cols].isna().sum()

In [ ]:
df_together_all  = pd.concat([df_c,df_together[numeric_cols]],axis = 1)

In [ ]:
df_together_all[numeric_cols].columns

In [ ]:
df_together_all = pd.concat([df_together_all,df_together['HasDetections']],axis = 1)

In [ ]:
df_together_all.isna().sum()

In [ ]:
X = df_together.iloc[:,:-1]
Y = df_together.iloc[:,-1]

In [ ]:
df_together_all.to_csv('fdata.csv',index = False)

In [ ]:
cx = X.corr()

In [ ]:
#sns.heatmap(cx)

In [ ]:
correlated_features = set()
for i in range(len(cx.columns)):
    for j in range(i):
        if abs(cx.iloc[i, j]) > 0.89:
            colname = cx.columns[i]
            correlated_features.add(colname)

In [ ]:
correlated_features

In [ ]:
#X.drop(columns = correlated_features,inplace = True)


In [ ]:
X.shape

In [ ]:
scaler = preprocessing.StandardScaler().fit(X)
X_scaled = scaler.transform(X)

In [ ]:
X_scaled.shape

In [ ]:
X_n = preprocessing.normalize(X_scaled)

In [ ]:
t = 0.3
train_inputs, val_inputs =  model_selection.train_test_split(X_n, test_size=t, random_state=0)
train_targets,val_targets =  model_selection.train_test_split(Y, test_size=t, random_state=0)   

In [ ]:
model = linear_model.LogisticRegression(max_iter=10000, class_weight = {0:1,1:1},C=1,solver='liblinear')
model.fit(train_inputs, train_targets)

In [ ]:
train_preds = model.predict(train_inputs)
#Y_pred2 = model.predict(test_inputs[numeric_cols])
print(classification_report(train_targets, train_preds))
val_preds = model.predict(val_inputs)
print(classification_report(val_targets, val_preds))
print(roc_auc_score(val_preds,val_targets))

In [ ]:
model2 = linear_model.LogisticRegression(max_iter=10000, class_weight = {0:10,1:1},C=100,solver='liblinear')
model2.fit(train_inputs, train_targets)
train_preds2 = model2.predict(train_inputs)
print(classification_report(train_targets, train_preds2))
val_preds2 = model2.predict(val_inputs)
print(classification_report(val_targets, val_preds2))
print(roc_auc_score(val_preds2,val_targets))

In [ ]:
# if we do not drop the corelated features and x= 10 encoding funct param, we get the roc_auc_score as 0.6369548155622827

In [ ]:
def remove_null_columns(cols,null_values,threshold):
    remove_list = []
    for i in range(len(cols)):
        if null_values[i] >= threshold:
            remove_list.append(cols[i])
    new_cols = cols.copy()
    for j in remove_list:  
        new_cols.remove(j)
    #print(len(new_cols),len(remove_list),len(cols))    
    return new_cols  

def encoding(x,cat_list,unique_list):
    label_list = []
    one_hot_list = []
    for i in range(len(cat_list)):
        if unique_list[i] >= x:
            label_list.append(cat_list[i])
        else:
             one_hot_list.append(cat_list[i])
                
            
    return label_list, one_hot_list 

def preprocessing_chunk(df):
    columns_list = []
    columns_list=df.columns
    numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
    categorical_cols = df.select_dtypes('object').columns.tolist()
    numeric_cols.remove('HasDetections')
    x =  list((df[numeric_cols].isna().sum()/df.shape[0])*100)
    new_numeric_cols=remove_null_columns(numeric_cols,x,30)
    y =  list((df[categorical_cols].isna().sum()/df.shape[0])*100)
    new_categorical_cols=remove_null_columns(categorical_cols,x,30)
    for i in new_numeric_cols:
        df[i].fillna(df[i].mean(),inplace =True)
    df_c = df[new_categorical_cols]
    df_c = df_c.drop(columns = 'MachineIdentifier')
    df_c = df_c.fillna(df_c.mode().iloc[0])
    col_list = df_c.columns
    unique_list=df_c.nunique()
    l_list,h_list = encoding(10,col_list,unique_list)
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
    encoder.fit(df_c[h_list])
    encoded_cols = list(encoder.get_feature_names(h_list))
    df_c[encoded_cols] = encoder.transform(df_c[h_list])
    lencoder = preprocessing.LabelEncoder()
    for i in l_list:
        df_c[i] = lencoder.fit_transform(df_c[i])
    df_c =df_c.drop(columns = h_list)
    df_together  = pd.concat([df_c,df[new_numeric_cols],df['HasDetections']],axis = 1)
    X = df_together.iloc[:,:-1]
    Y = df_together.iloc[:,-1]
    return df_together

In [ ]:
df_test = pd.read_csv("/kaggle/input/save-the-attack-contest/test_data.csv") 

In [ ]:
mi = df_test['MachineIdentifier']

In [ ]:
df

In [ ]:
df_together  = pd.concat([df,df_test],axis =0)

In [ ]:
df_new = preprocessing_chunk(df_together)
df_train = df_new.iloc[:100000,:]
df_test  = df_new.iloc[100000:,:]

In [ ]:
X_train = df_train.iloc[:,:-1]
Y_train = df_train.iloc[:,-1]
X_test = df_test.iloc[:,:-1]


In [ ]:
scaler = preprocessing.StandardScaler().fit(X_train)
X_scaled_train = scaler.transform(X_train)
scaler2 = preprocessing.StandardScaler().fit(X_test)
X_scaled_test = scaler2.transform(X_test)

In [ ]:
X_n_train = preprocessing.normalize(X_scaled_train)
X_n_test = preprocessing.normalize(X_scaled_test)

In [ ]:
model2 = linear_model.LogisticRegression(max_iter=10000, class_weight = {0:10,1:1},C=100,solver='liblinear')
model2.fit(X_n_train, Y_train)
train_preds2 = model2.predict(X_n_train)
print(classification_report(Y_train, train_preds2))
print(roc_auc_score(train_preds2,Y_train))


In [ ]:
val_preds2 = model2.predict(X_n_test)

In [ ]:
model3 = linear_model.LogisticRegression(max_iter=10000, class_weight = {0:1,1:1},C=1,solver='liblinear')
model3.fit(X_n_train, Y_train)
train_preds3 = model3.predict(X_n_train)
print(classification_report(Y_train, train_preds3))
print(roc_auc_score(train_preds3,Y_train))#0.6200320101127694

In [ ]:
val_preds3 = model3.predict(X_n_test)

In [ ]:
submission = pd.DataFrame({
        "MachineIdentifier": mi,
        "HasDetections": pd.Series(val_preds3)
    })

submission.to_csv('submission.csv', index=False)

In [ ]:
'''
from sklearn.svm import SVC
svclassifier = SVC(kernel='linear', C = 0.1)
svclassifier.fit(train_inputs, train_targets)
train_preds3 = svclassifier.predict(train_inputs)
print(classification_report(train_targets, train_preds3))
val_preds3 = svclassifier.predict(val_inputs)
print(classification_report(val_targets, val_preds3))
print(roc_auc_score(val_preds3,val_targets))
'''

In [ ]:
'''
from sklearn.svm import SVC
svclassifier = SVC(kernel='poly', degree=2)
svclassifier.fit(train_inputs, train_targets)
train_preds4 = svclassifier.predict(train_inputs)
print(classification_report(train_targets, train_preds4))
val_preds4 = svclassifier.predict(val_inputs)
print(classification_report(val_targets, val_preds4))
print(roc_auc_score(val_preds4,val_targets))
'''

In [ ]:

#from sklearn.svm import SVC
#svclassifier = SVC(kernel="rbf", C=0.1, gamma=1)
#svclassifier.fit(train_inputs, train_targets)
#train_preds5 = svclassifier.predict(train_inputs)
#print(classification_report(train_targets, train_preds5))
#val_preds5 = svclassifier.predict(val_inputs)
#print(classification_report(val_targets, val_preds5))
#print(roc_auc_score(val_preds5,val_targets))


In [ ]:

#from sklearn.svm import SVC
#svclassifier = SVC(kernel="linear", C=0.1, class_weight={0:1,1:1})
#svclassifier.fit(train_inputs, train_targets)
#train_preds6 = svclassifier.predict(train_inputs)
#print(classification_report(train_targets, train_preds6))
#val_preds6 = svclassifier.predict(val_inputs)
#print(classification_report(val_targets, val_preds6))
#print(roc_auc_score(val_preds6,val_targets))
